In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [1]:
PATH = "data/Motor___MDS-UPDRS/"
PATH_DAT = "./"

In [2]:
import pydicom
import glob
import os
import numpy

In [5]:
up3 = pd.read_csv(f'{PATH}MDS_UPDRS_Part_III__Post_Dose_.csv')
up3['TOTAL'] = (up3.iloc[:,8:42]).sum(axis=1)

In [6]:
#up3.loc[up3['EVENT_ID']=='BL',"EVENT_ID"] = 'SC'

In [7]:
up3 = up3[~up3["PAG_NAME"].str.contains('A')]

In [8]:
up3['index'] = up3["PATNO"].map(str)+up3["EVENT_ID"].map(str)
#up3.set_index('index',inplace=True)
up3 = up3[['PATNO','TOTAL','index']]

In [9]:
up3.loc[up3['PATNO']=='3762']

,PATNO,TOTAL,index
3,3762,11.0,3762BL
392,3762,13.0,3762V01
655,3762,29.0,3762SC
1854,3762,22.0,3762V03
2128,3762,22.0,3762V02
3032,3762,15.0,3762V09
3335,3762,18.0,3762V08
5720,3762,17.0,3762V04
5916,3762,25.0,3762V05
6460,3762,30.0,3762V06


In [10]:
dat = pd.read_csv(f'{PATH_DAT}PPMI.csv')

In [11]:
#Change Visit code to standard code
dat.loc[dat['Visit']==0,"Visit"] = 'SC'
dat.loc[dat['Visit']==5,"Visit"] = 'V04'
dat.loc[dat['Visit']==7,"Visit"] = 'V06'
dat.loc[dat['Visit']==9,"Visit"] = 'V09'
dat.loc[dat['Visit']==11,"Visit"] = 'V10'
dat.loc[dat['Visit']==13,"Visit"] = 'V13'
dat.loc[dat['Visit']==21,"Visit"] = 'V21'
dat.loc[dat['Visit']==90,"Visit"] = 'U01'
dat.loc[dat['Visit']==91,"Visit"] = 'U02'
dat.loc[dat['Visit']==92,"Visit"] = 'U03'

In [12]:
dat["Visit"].unique()

array(['SC', 'V04', 'V06', 'V10', 'V21', 'V09', 'U01', 'U03', 'V13', 'U02'], dtype=object)

In [13]:
dat['index'] = dat["Subject"].map(str)+dat["Visit"].map(str)
dat = dat[['Group','index','Image Data ID']]
#dat.set_index('index',inplace=True)

In [14]:
dat_analysis = pd.read_csv('data/DaTSCAN/DATScan_Analysis.csv')

In [15]:
dat_analysis['EVENT_ID'].unique()

array(['SC', 'V10', 'V06', 'V04', 'ST', 'U01', 'U02'], dtype=object)

In [16]:
combine = pd.merge(up3, dat, how='inner', on='index')

In [17]:
combine.head()

,PATNO,TOTAL,index,Group,Image Data ID
0,3154,41.0,3154V04,PD,355939
1,3954,13.0,3954SC,PD,340410
2,3150,27.0,3150V04,PD,355938
3,3650,15.0,3650V04,PD,355955
4,3309,24.0,3309SC,PD,339895


In [18]:
lstFilesDCM =[]
for i in combine['Image Data ID']:
    for file in glob.glob(f'{PATH_DAT}PPMI*/**/Reconstructed_DaTSCAN/**/**/*.dcm'):
        file = file.split("_")[13].strip('I').strip('.dcm')
        if i == file:
            lstFilesDCM.append(file)

AttributeError: 'function' object has no attribute 'glob'

In [21]:
lstFilesDCM =[]
for file in glob(f'{PATH_DAT}PPMI*/**/Reconstructed_DaTSCAN/**/**/*.dcm'):
    #file = file.split("_")[13].strip('I').strip('.dcm')
    lstFilesDCM.append(file)

In [22]:
len(lstFilesDCM)

1569

In [23]:
stripLst = []
for x in lstFilesDCM:
    x = x.split("_")[13].strip('I').strip('.dcm')
    stripLst.append(x)

In [24]:
combine = combine.set_index('Image Data ID')
for index, rows in combine.iterrows():
    for i in lstFilesDCM:
        j = i.split("_")[13].strip()
        j = int(j[1:7])
        if index == j:
            combine.loc[combine.index == j, "Filenames"] = i
combine

,PATNO,TOTAL,index,Group,Filenames
Image Data ID,,,,,
355939,3154,41.0,3154V04,PD,./PPMI 8/3154/Reconstructed_DaTSCAN/2011-12-06...
340410,3954,13.0,3954SC,PD,./PPMI 3/3954/Reconstructed_DaTSCAN/2012-01-04...
355938,3150,27.0,3150V04,PD,./PPMI/3150/Reconstructed_DaTSCAN/2011-11-30_1...
355955,3650,15.0,3650V04,PD,./PPMI 7/3650/Reconstructed_DaTSCAN/2011-12-07...
339895,3309,24.0,3309SC,PD,./PPMI 3/3309/Reconstructed_DaTSCAN/2011-12-07...
358125,3417,25.0,3417V04,PD,./PPMI 8/3417/Reconstructed_DaTSCAN/2012-03-28...
358149,3601,32.0,3601V04,PD,./PPMI 8/3601/Reconstructed_DaTSCAN/2012-03-27...
342177,3120,26.0,3120SC,PD,./PPMI 9/3120/Reconstructed_DaTSCAN/2012-04-17...
358119,3107,2.0,3107V04,PD,./PPMI 7/3107/Reconstructed_DaTSCAN/2012-03-21...


In [25]:
combine = combine[combine.index.isin(stripLst)]

In [26]:
combine.to_csv('combine.csv')